# HW2: Probability, Naive Bayes, and Linear Regression
**Andrew Corum, Jo Youngheun**

Please grade this submission. Thanks!

## Question 1
### Code for building and evaluating Naive Bayes models:

In [1]:
import pandas as pd
import numpy as np
import math

# For splitting data and computing confusion matrix
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import confusion_matrix

# For set of english stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/amcorum/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def build_model(X, y, laplace=False, ignore_stop_words=False):
    '''
    Calculate required values for Naive Bayes classifier,
    so we can later compute priors and liklihoods
    
    Parameters:
        X (np.array): messages
        y (np.array): message label (0 or 1)
        laplace (bool): Whether or not to use laplace smoothing
    Returns:
        model (dict):
            {
                'prior': {label: log(Prob(label))}
                    log-prior probabilities of each label
                'label_word_count': {label: log(count(label))}
                    log-count of total number of words found in messages with each label
                'word_counts': {word: {label: count}}
                    for each unique word, number of times it appears in messages with each label
            }
    '''
    prior = {l: 0 for l in [0,1]}
    label_word_count = {l: 0 for l in [0,1]}
    word_counts = {}
    # Loop through each row in the data
    for msg, label in zip(X, y):
        # Keep track of number of occurances of the given label
        prior[label] += 1
        # Also count number of total words found in the given label
        label_word_count[label] += len(msg.split())
        
        # For each word in the message, total up how many times it appears in a spam or non-spam message
        for word in msg.split():
            # If we are ignoring stop words, then don't count a stop word...
            if ignore_stop_words and word in stop_words:
                label_word_count[label] -= 1
                continue
            # int(laplace) tells us to initialize new words with count==1 when doing laplace smoothing
            if word not in word_counts: word_counts[word] = {l: int(laplace) for l in [0,1]}
            word_counts[word][label] += 1
            
    # Divide lable counts by total number of rows, to get prior probability. Then take log
    prior = {l: math.log(prior[l])-math.log(len(X)) for l in prior}
    # Take log of total word counts (since we will use log in model_predict() anyways)
    label_word_count = {l: math.log(label_word_count[l]) for l in label_word_count}
    
    return {'prior': prior, 'label_word_count': label_word_count, 'word_counts': word_counts}

def model_predict(model, X, laplace=False, ignore_stop_words=False):
    '''
    Using a pre-computed model (priors, total label word counts, and word counts per lablel),
    predict 0,1 spam labels given input messages.
    
    Parameters:
        model (dict):
            {
                'prior': {label: log(Prob(label))}
                    log-prior probabilities of each label
                'label_word_count': {label: log(count(label))}
                    log-count of total number of words for each label
                'word_counts': {word: {label: count}}
                    count of each label for each unique word
            }
        X (np.array): messages
        laplace (bool): Whether or not to use laplace smoothing
    Returns:
        y_pred (list): list of label estimates
    '''
    y_pred = []
    for msg in X:
        # Start with prior probability
        prob0, prob1 = model['prior'][0], model['prior'][1]
        # Multiply prior by liklihood of each word given the label
        for word in msg.split():
            # If ignoring stop words, ignore the stop word...
            if ignore_stop_words and word in stop_words: continue
            # If using laplace smoothing, set word count to 1... otherwise skip the extra word
            if word not in model['word_counts']:
                if laplace: model['word_counts'][word] = {0: 1, 1: 1}
                else: continue
            word_count = model['word_counts'][word]
            # Again, if word not found in training set and not using laplace, skip the word
            if not laplace and (model['word_counts'][word][0] == 0 or model['word_counts'][word][1] == 0):
                continue
            # word_count still needs to be put into log form
            prob_word = [math.log(word_count[i]) for i in [0,1]]
            
            # logP(word | label) := log(#times <word> appears in <label> messages / total #words in <label> messages)
            prob0 += prob_word[0] - model['label_word_count'][0]
            prob1 += prob_word[1] - model['label_word_count'][1]
            
            # If laplace smoothing, need to divide liklihood by number of features (ie. number of words in the input message)
            if laplace:
                prob0 -= math.log(len(msg.split()))
                prob1 -= math.log(len(msg.split()))
                
        y_pred.append(0 if prob0 >= prob1 else 1)
    return y_pred

In [3]:
def print_stats(conf):
    '''
    Given a confusion matrix, compute and print accuracy, precision, recall, and specificity
    '''
    print("  Accuracy: {}\n  Precision: {}\n  Recal: {}\n  Specificity: {}".format(
        (conf[0][0]+conf[1][1])/sum(sum(conf)),
        conf[1][1]/(conf[1][1] + conf[1][0]),
        conf[1][1]/(conf[1][1] + conf[0][1]),
        conf[0][0]/(conf[0][0] + conf[1][0])
    ))

def evaluate_naive_bayes(X, y, laplace=False):
    '''
    Evaluate naive bayes classifier using 10-fold cross validation
    Print out performance stats and confusion matrix for each fold
    Print out average performance stats
    
    Parameters:
        X (np.array): messages
        y (np.array): labels (0 or 1) 
        laplace (bool): Whether or not to use laplace smoothing
    '''
    fold, total_conf = 0, [[0,0],[0,0]]
    # Within train data, use 10-fold split, stratified based on spam label
    trainKFold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    for train_index, test_index in trainKFold.split(X, y):
        fold += 1
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]
        
        # Build model on train data
        model = build_model(X_train, y_train, laplace)
        # Predict based on validation data
        y_pred = model_predict(model, X_val, laplace)
        # Compare predicted labels with true labels
        conf = confusion_matrix(y_val, y_pred)
        total_conf += conf
        
        print("Fold {}:".format(fold))
        print_stats(conf)
        print("  Confusion matrix:\n{}".format(conf))
    
    print("Average over all folds:")
    print_stats(total_conf/10)

### Naive Bayes training performance (ignoring words not in training data)

In [4]:
# Get 'message.csv' data
msgdf = pd.read_csv("./message.csv")
X, y = msgdf['Message'].to_numpy(), msgdf['Label'].to_numpy()

# Split data into train/test, stratified based on spam label
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, stratify=y)

In [5]:
evaluate_naive_bayes(X_train, y_train)

Fold 1:
  Accuracy: 0.9641255605381166
  Precision: 0.8833333333333333
  Recal: 0.8548387096774194
  Specificity: 0.9817708333333334
  Confusion matrix:
[[377   9]
 [  7  53]]
Fold 2:
  Accuracy: 0.968609865470852
  Precision: 0.8833333333333333
  Recal: 0.8833333333333333
  Specificity: 0.9818652849740933
  Confusion matrix:
[[379   7]
 [  7  53]]
Fold 3:
  Accuracy: 0.9753363228699552
  Precision: 0.9333333333333333
  Recal: 0.8888888888888888
  Specificity: 0.9895561357702349
  Confusion matrix:
[[379   7]
 [  4  56]]
Fold 4:
  Accuracy: 0.9641255605381166
  Precision: 0.8166666666666667
  Recal: 0.9074074074074074
  Specificity: 0.9719387755102041
  Confusion matrix:
[[381   5]
 [ 11  49]]
Fold 5:
  Accuracy: 0.9775784753363229
  Precision: 0.9666666666666667
  Recal: 0.8787878787878788
  Specificity: 0.9947368421052631
  Confusion matrix:
[[378   8]
 [  2  58]]
Fold 6:
  Accuracy: 0.9775784753363229
  Precision: 0.95
  Recal: 0.890625
  Specificity: 0.9921465968586387
  Confusion 

**Summary:**

This naive bayes classifier is quite good at identifying spam messages.
The classifier works by treating a message $x$ as an input with $n$ features (where $n$ is the number of words in $x$). Then it performs maximum a posteriori (MAP) estimation to find the 0/1 label (0=='not spam', 1=='spam') that best represents the message. This is done by finding the $y\in\{0,1\}$ label that maximizes the probability $P(y|x) \propto P(y)\Pi_{i=0}^{n}{P(x_i | y)}$. Although, since the product of the liklihood probabilities quickly approach zero, the code above uses log-liklihood: $\log P(y|x) \propto \log P(y)\Sigma_{i=0}^{n}{\log P(x_i | y)}$. To calculate the prior $P(y=y')$, we can just determine the number of messages that are labeled $y'$ and divide by the total number of messages. To calculate the liklihood $P(x=x_i|y=y')$, we can use the equation $P(x=x_i|y=y')=\frac{n_{x_i}}{n_{y'}}$. In this equation, $n_{x_i}$ is the number of times the word $x_i$ appears in messages labeled $y'$, and $n_{y'}$ is the total number of words in messages labeled $y'$. However, words with zero probability are completely ignored, as they have undefined log values.

*Note*: Rather than keeping track of an entire $N\times M$ matrix of unique words, as suggested in the hw instructions, I use 3 dictionaries. This may not save much space, but I found it makes the liklihood computation a little simpler once you get to the `model_predict()` function.

### Naive Bayes training performance (with laplace smoothing)

In [6]:
evaluate_naive_bayes(X_train, y_train, laplace=True)

Fold 1:
  Accuracy: 0.9663677130044843
  Precision: 0.9666666666666667
  Recal: 0.8169014084507042
  Specificity: 0.9946666666666667
  Confusion matrix:
[[373  13]
 [  2  58]]
Fold 2:
  Accuracy: 0.952914798206278
  Precision: 0.9333333333333333
  Recal: 0.7671232876712328
  Specificity: 0.9892761394101877
  Confusion matrix:
[[369  17]
 [  4  56]]
Fold 3:
  Accuracy: 0.9798206278026906
  Precision: 0.9833333333333333
  Recal: 0.8805970149253731
  Specificity: 0.9973614775725593
  Confusion matrix:
[[378   8]
 [  1  59]]
Fold 4:
  Accuracy: 0.9663677130044843
  Precision: 0.95
  Recal: 0.8260869565217391
  Specificity: 0.9920424403183024
  Confusion matrix:
[[374  12]
 [  3  57]]
Fold 5:
  Accuracy: 0.9618834080717489
  Precision: 0.9833333333333333
  Recal: 0.7866666666666666
  Specificity: 0.9973045822102425
  Confusion matrix:
[[370  16]
 [  1  59]]
Fold 6:
  Accuracy: 0.968609865470852
  Precision: 0.9833333333333333
  Recal: 0.8194444444444444
  Specificity: 0.9973262032085561
  C

**Summary:**

In this Naive Bayes classifier, we employ laplace smoothing.
The classifier works similarly to the previous model, but the liklihood is calculated differently. Instead of ignoring words that did not appear in the training set, these words are given a small likihood (by assuming they appeared once in the training set). The new liklihood equation looks like $P(x=x_i|y=y')=\frac{n_{x_i}+1}{n_{y'}+N}$. In this equation, $n_{x_i}$ is the number of times the word $x_i$ appears in messages labeled $y'$, $n_{y'}$ is the total number of words in messages labeled $y'$, and $N$ is the total number of features (ie words) appearing in the input message. This approach makes more sense from a theoretical standpoint, since "ignoring" words is actually treating them as having a liklihood of $1$. However, for this data set, it appears the laplace smoothing method results in a slightly less accurate classifier. Although, it has better precision and specificity, meaning that the spam classifications are more likely to be correct (than without smoothing).

### Naive Bayes performance on test data

In [7]:
# Train models on all training data, test on remaining 20%

# No laplace smoothing. Keep stop words
model = build_model(X_train, y_train)
y_pred = model_predict(model, X_test)
conf = confusion_matrix(y_test, y_pred)
print("No laplace smoothing:")
print_stats(conf)
print("  Confusion matrix:\n{}".format(conf))

# With laplace smoothing. Keep stop words
model = build_model(X_train, y_train, laplace=True)
y_pred = model_predict(model, X_test, laplace=True)
conf = confusion_matrix(y_test, y_pred)
print("With laplace smoothing:")
print_stats(conf)
print("  Confusion matrix:\n{}".format(conf))

# No laplace smoothing. Remove stop words
model = build_model(X_train, y_train, laplace=False, ignore_stop_words=True)
y_pred = model_predict(model, X_test, laplace=False, ignore_stop_words=True)
conf = confusion_matrix(y_test, y_pred)
print("No laplace smoothing and ignoring stop words:")
print_stats(conf)
print("  Confusion matrix:\n{}".format(conf))

# With laplace smoothing. Remove stop words
model = build_model(X_train, y_train, laplace=True, ignore_stop_words=True)
y_pred = model_predict(model, X_test, laplace=True, ignore_stop_words=True)
conf = confusion_matrix(y_test, y_pred)
print("With laplace smoothing and ignoring stop words:")
print_stats(conf)
print("  Confusion matrix:\n{}".format(conf))

No laplace smoothing:
  Accuracy: 0.9623318385650225
  Precision: 0.9060402684563759
  Recal: 0.8282208588957055
  Specificity: 0.9852941176470589
  Confusion matrix:
[[938  28]
 [ 14 135]]
With laplace smoothing:
  Accuracy: 0.9614349775784753
  Precision: 0.959731543624161
  Recal: 0.7944444444444444
  Specificity: 0.9935828877005347
  Confusion matrix:
[[929  37]
 [  6 143]]
No laplace smoothing and ignoring stop words:
  Accuracy: 0.9524663677130045
  Precision: 0.912751677852349
  Recal: 0.7727272727272727
  Specificity: 0.9861554845580405
  Confusion matrix:
[[926  40]
 [ 13 136]]
With laplace smoothing and ignoring stop words:
  Accuracy: 0.95695067264574
  Precision: 0.959731543624161
  Recal: 0.772972972972973
  Specificity: 0.9935483870967742
  Confusion matrix:
[[924  42]
 [  6 143]]


**Summary**:

We originally created an 80-20 stratified test-train split in our data. The code above shows the results of training the model on the full 80% of data, then testing on the remaining 20%. We can see that again the laplace smoothing gives us much better precision and specificity, at a small cost to the recall and accuracy. For this data set, it seems that implementing laplace smoothing makes the model less sensitive to messages that may be "spam" messages... but this may be helpful if we don't want our model to create too many false positives.

We also test here two models that ignore "stop words". These stop words are a corpus of words that appear frequently in the english dictionary. They are also words that do not carry much meaning, such as "in", "an", "the", "about", etc.
Since stop words do not have much semantic influence on a sentence, they also should not influence whether or not a message is "spam". However, we can see that the models that ignore stop words actually perform slightly worse. Again, this probably has to do with our particular data set. It seems that the normal model is able to find some correlation between stop words and spam messages; hence removing stop words hurts our classifier's performance. It might be interesting to try to see which stop words had the highest liklihoods within spam/not-spam messages. There may be a couple stop words in the corpus that actually carry some semantic weight in regards to this classifcation process.

## Resources used in Question 1
* Pandas documentation for references on DataFrames (https://pandas.pydata.org/docs/)
* Scikit Learn documentation and sample code for [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), [StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html), and [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)
* Referenced Wikipedia definitions of precision, recall, and specificity: https://en.wikipedia.org/wiki/Precision_and_recall, https://en.wikipedia.org/wiki/Sensitivity_and_specificity
* GeeksForGeeks tutorial on removing stop words: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
* Referenced my (Andrew Corum's) previously completed B551 assignment where I created a Naive Bayes classifier (from Fall 2020). I did not copy any of my old code, but I did use it as a reference. Some of my structure is similar. For example, in the B551 assignment I had the idea to save the model as three dictionaries (`prior`, `label_word_count`, and `word_count`). But again, I only used this as a reference, and it was all code that was written by me.